# Enumerate Facet inequalities for a given number of inputs and binary outputs
Here I want to implement an algorithm, to find all facet inequalities for a given number of inputs and binary outputs.
This follows the construction of *Algorithm 1* in the paper [*Bell inequalities from no-signalling distribution*, *Cope & Colbeck (2019)*](https://arxiv.org/abs/1812.10017).

First we set the number of inputs and outputs and obtain the extremal points for the specific NS polytope.

In [15]:
from utils import extremal_ns_binary_vertices, get_deterministic_behaviors, get_allowed_relabellings
import numpy as np
# set inputs / outputs
inputs_a = range(4)
inputs_b = range(4)
outputs = range(2)

# get extremal points
extremals = extremal_ns_binary_vertices(inputs_a, inputs_b, outputs)

# get deterministic points
dets = get_deterministic_behaviors(inputs_a, inputs_b, outputs)

# get allowed relabellings
allowed_relabellings = get_allowed_relabellings(inputs_a, inputs_b, outputs)

# options for local weight optimizer (bland is only needed for higher dimensions (m_a , m_b > 4)
options = {"disp": False, "maxiter": 5000, "bland": True}

In [ ]:
from utils import find_local_weight, facet_inequality_check, check_diff_repr_same_ineq, check_equiv_relabel
from itertools import product
epsilons = np.linspace(0,2/3, num=4)
epsilons = [1/3]

# list of all facets
facets = []
# iterate through extremals
for z, e in enumerate(extremals):
    print('{} / {} : num of facets: {}'.format(z,len(extremals),len(facets)))
    # find the local weight for the extremal behavior
    opt, bell_expression = find_local_weight(e, dets, method='simplex', options=options)
    # check that there is no local part, as it is an extremal point
    assert np.abs(bell_expression @ e) < 1e-6, 'local weight of extremal is not zero: {}'.format(np.abs(bell_expression @ e))
    # find all equalizing local behaviors for the inequality bell_expression @ d >= 1
    is_facet, bell_expression, eq_dets = facet_inequality_check(dets, bell_expression, len(inputs_a), len(inputs_b), len(outputs))
    # check if we found a new facet, by increasing complexity
    if not is_facet: continue
    # check if other representation is already in facets
    check = [check_diff_repr_same_ineq(bell_expression,f,dets) for f in facets]
    if True in check: continue
    # check if relabelled version is already in facets
    check = [check_equiv_relabel(bell_expression, f, allowed_relabellings) for f in facets]
    if True in check: continue
    # it's a new facet -> append to all facets
    facets.append(bell_expression)
    # iterate through epsilons
    for epsilon in epsilons:
        # define new behaviors based on epsilon
        for j,k in product(range(eq_dets.shape[0]),repeat=2):
            # need distinct j,k
            if j == k:
                continue
            # form new probability distribution
            e_new = (1-3*epsilon/2)*e + epsilon*eq_dets[j] + epsilon / 2 * eq_dets[k]
            # do the facet check again
            opt, bell_expression = find_local_weight(e_new, dets,method='simplex', options=options)
            is_facet, bell_expression, eq_dets_new = facet_inequality_check(dets, bell_expression, len(inputs_a), len(inputs_b), len(outputs))
            # do the same checks as above, could be done in a better way
            if not is_facet: continue
            check = [check_diff_repr_same_ineq(bell_expression,f,dets) for f in facets]
            if True in check: continue
            check = [check_equiv_relabel(bell_expression, f, allowed_relabellings) for f in facets]
            if True in check: continue
            facets.append(bell_expression)
print('Number of facet classes found: {}'.format(len(facets)))

0 / 256 : num of facets: 0


Now we have found the number of classes of facets for a specific number of inputs and outputs.
